In [41]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score

In [42]:
# params
params_arr = [(1000, 0, 10, 5),
              (1000, 0, 15, 5),
              (1000, 0, 20, 5),
              (1000, 0, 10, 10),
              (1000, 0, 10, 7)
             ]
params = {
    "n_estimators": 1000,
    "random_state": 0,
    "max_depth": 10,
    "min_samples_split": 2,
    "min_samples_leaf": 80,
    "n_jobs": 40,
    "max_features": 5,
    "oob_score": True
}

In [43]:
# read all data from train
train_data = [pd.read_csv(f'./data/train{i+1}.csv', header=None) for i in range(5)]
train_label = [pd.read_csv(f'./data/label{i+1}.csv', header=None) for i in range(5)]

x = np.vstack([np.array(train_data[i]) for i in range(5)])
y = np.vstack([np.array(train_label[i]) for i in range(5)])
print(x.shape, y.shape)

(10000004, 13) (10000004, 1)


In [44]:
for i in range(len(params_arr)):
    params["max_depth"] = params_arr[i][2]
    params["max_features"] = params_arr[i][3]
    rf = RandomForestRegressor(**params)
    rf.fit(x, y)
    print(rf.oob_score_)

/home/lab/anaconda3/envs/pytorch/lib/python3.6/site-packages/ipykernel_launcher.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """


0.15228532654914806


/home/lab/anaconda3/envs/pytorch/lib/python3.6/site-packages/ipykernel_launcher.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """


0.15746375262990997


/home/lab/anaconda3/envs/pytorch/lib/python3.6/site-packages/ipykernel_launcher.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """


0.15882076159079062


/home/lab/anaconda3/envs/pytorch/lib/python3.6/site-packages/ipykernel_launcher.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """


0.15254845836285247


/home/lab/anaconda3/envs/pytorch/lib/python3.6/site-packages/ipykernel_launcher.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """


ValueError: max_features must be in (0, n_features]

In [ ]:
params["max_depth"] = params_arr[2][2]
params["max_features"] = params_arr[2][3]
rf = RandomForestRegressor(**params)
rf.fit(x, y)

/home/lab/anaconda3/envs/pytorch/lib/python3.6/site-packages/ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.


In [ ]:
# read test data
test = [pd.read_csv(f'./data/test{i + 1}.csv', header=None) for i in range(6)]
test_x = np.vstack([np.array(test[i]) for i in range(6)])

In [ ]:
arr = [len(test[i]) for i in range(6)]
arr

In [ ]:
# predict and write
prediction = rf.predict(test_x)
output = pd.DataFrame(data = {'id': list(range(1, len(prediction)+1)), 'Predicted': prediction})
output.to_csv('result.csv', index=False)